# Importing required libraries

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
import scipy
from keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import cross_validate

from sklearn.metrics import f1_score,log_loss,precision_score,recall_score,make_scorer,accuracy_score

# Importing a saved dataset so that all the four models are given the same input

In [4]:
data1 = pd.read_csv("word2vec_movies.csv")
data1

,Unnamed: 0,Unnamed: 0.1,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot,sum_w2v,avg_w2v
0,5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...,[ 2.556839 3.1699066 2.5140228 0.45...,[ 0.03551165 0.04402648 0.03491698 0.006299...
1,19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...,[ 1.83398438e+00 1.34423828e+00 -3.68164062e-...,[ 7.33593777e-02 5.37695326e-02 -1.47265624e-...
2,40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...,[ 15.684422 9.671326 4.4506836 11.86...,[ 0.07842211 0.04835663 0.02225342 0.059335...
3,15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...,[ 4.185997 0.38246155 -0.7231293 5.50...,[ 0.05507891 0.00503239 -0.00951486 0.072433...
4,30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...,[ 6.78332520e+00 2.71803284e+00 -7.62779236e-...,[ 7.98038244e-02 3.19768563e-02 -8.97387322e-...
...,...,...,...,...,...,...,...,...,...,...
4174,7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...,[ 6.591466 12.522953 3.6369324 7.44...,[ 0.03009802 0.05718243 0.016607 0.034015...
4175,7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...,[ 3.06790924e+00 5.40142250e+00 7.32398033e+...,[ 7.86643382e-03 1.38498014e-02 1.87794361e-...
4176,31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...,[ 2.1972656e-01 4.2459106e-01 -5.4833221e-01 ...,[ 1.15645556e-02 2.23468989e-02 -2.88595911e-...
4177,15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...,[ 16.906288 16.096968 24.503977 21.56...,[ 0.02981709 0.02838971 0.04321689 0.038034...


In [5]:
genres = [] 

for i in data1['genre']: 
  genres.append(list(json.loads(i).values()))

data1['genre_list'] = genres

genres_list = sum(genres,[])
print("No of Genres: ",len(set(genres_list)))

No of Genres:  282


In [6]:
multilabel_binarizer1 = MultiLabelBinarizer()
multilabel_binarizer1.fit(data1['genre_list'])
y1 = multilabel_binarizer1.transform(data1['genre_list'])
y1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
vectorizer1 = CountVectorizer()
x1 = vectorizer1.fit_transform(data1['filtered_plot'])
print(x1.shape,y1.shape)

(4179, 41619) (4179, 282)


# Initiating the Model and using cross-validation

In [10]:
lr1 = LogisticRegression()
clf1 = OneVsRestClassifier(lr1)

In [11]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'micro'),
           'recall': make_scorer(recall_score, average = 'micro'),
           'f1': make_scorer(f1_score, average = 'micro'),
           'log_loss': make_scorer(log_loss)}

In [12]:
scores = cross_validate(clf1, x1, y1, cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([122.99424481, 118.96698213, 122.60647678, 125.98528242,
        124.44338489]),
 'score_time': array([0.29118395, 0.24434686, 0.26058841, 0.27238607, 0.27301264]),
 'test_accuracy': array([0.03947368, 0.05143541, 0.04186603, 0.04904306, 0.03353293]),
 'train_accuracy': array([0.99282082, 0.99311995, 0.99072689, 0.99192342, 0.99282297]),
 'test_precision': array([0.5046225 , 0.50276625, 0.52102102, 0.51408987, 0.49040512]),
 'train_precision': array([1., 1., 1., 1., 1.]),
 'test_recall': array([0.21252433, 0.2512094 , 0.2232229 , 0.21767172, 0.22794846]),
 'train_recall': array([0.9970324 , 0.99756474, 0.99603437, 0.99653236, 0.99671754]),
 'test_f1': array([0.29908676, 0.33502304, 0.31254222, 0.30584504, 0.31123139]),
 'train_f1': array([0.99851399, 0.99878088, 0.99801325, 0.99826317, 0.99835607]),
 'test_log_loss': array([79.00897441, 72.93769778, 79.36077515, 76.90369797, 76.67711911]),
 'train_log_loss': array([5.66207819, 5.78008483, 5.74850482, 5.68642467, 5.73

In [13]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Train Log Loss is: ",scores['train_log_loss'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")
print("The Test Log Loss is: ",scores['test_log_loss'].mean())
print("\n")

The Fit Time is:  122.9992742061615


The Score Time is:  0.26830358505249025


The Train Accuracy score is:  0.9922828104716418


The Train F1 score is:  0.9983854735575697


The Train Precision score is:  1.0


The Train Recall score is:  0.9967762823723441


The Train Log Loss is:  5.722458094408212


The Test Accuracy score is:  0.04307022318998367


The Test F1 score is:  0.31274569039507694


The Test Precision score is:  0.5065809513388303


The Test Recall score is:  0.22651536349678564


The Test Log Loss is:  76.97765288540411


